# Taxes & Transfers Objects and Modifications of the Policy Environment

GETTSIM's design allows you to go beyond the depiction of the current (or historical)
tax and transfer system. Analyzing counterfactual reform scenarios, ranging from small
changes of certain parameters of the tax and transfer system, to the introduction of
large-scale reforms, is a common use case.

This tutorial showcases how to modify the calculation of taxes and transfers when using
GETTSIM.

Here, we focus mainly on small reforms to the calculation of income taxes. We pick this
example because it is a fairly complex system that uses (almost) the entire range of
objects TTSIM offers.

## Status Quo

Before modifying taxes and transfers, it's important to understand how GETTSIM
represents the current tax and transfer system. The core of GETTSIM's implementation is
the **policy environment**.

### What is a Policy Environment?

A policy environment is a nested dictionary that holds all the parameters and functions
needed to calculate taxes and transfers for a given policy date. Think of it as a
complete snapshot of the tax and transfer system at a particular point in time.

### The Three Types of Objects

The policy environment contains three main categories of objects:

1. **Column Objects** (`ColumnObjects`): These work with data columns - either input
   data you provide or results computed by previous functions. They handle the actual
   calculations and data processing.

2. **Parameter Objects** (`ParamObjects`): These store the parameters and constants
   used in calculations, such as tax rates, benefit amounts, or thresholds.

3. **Parameter Functions** (`ParamFunctions`): These process parameters which cannot be
   used directly by the column objects.

### Getting Started

After the standard imports, the first step in modifying taxes and transfers is to create
the base policy environment for the date you want to work with.

In [ ]:
from __future__ import annotations

import numpy as np

from gettsim import InputData, MainTarget, copy_environment, main

In [ ]:
status_quo_environment = main(
    main_target=MainTarget.policy_environment,
    policy_date_str="2025-01-01",
    backend="numpy",
)

We also create some input data in order to verify how our modifications affect the
output. The following input data is required to compute the amount of income tax when
assuming parental leave benefits (i.e. `('elterngeld', 'betrag_m')`), pensions (i.e.
`('sozialversicherung', 'rente', 'altersrente', 'betrag_m')`, `('sozialversicherung',
'rente', 'erwerbsminderung', 'betrag_m')`) and unemployment benefits (i.e.
`('sozialversicherung', 'arbeitslosen', 'betrag_m')`) are fixed at some value.

In [ ]:
INPUT_DATA_TREE = {
    "alter": np.array([40, 40, 5]),
    "arbeitsstunden_w": np.array([40, 40, 0]),
    "behinderungsgrad": np.array([0, 60, 0]),
    "einkommensteuer": {
        "abzüge": {
            "beitrag_private_rentenversicherung_m": np.array([120, 120, 0]),
            "kinderbetreuungskosten_m": np.array([0.0, 0.0, 120.0]),
            "p_id_kinderbetreuungskostenträger": np.array([-1, -1, 0]),
        },
        "einkünfte": {
            "aus_forst_und_landwirtschaft": {"betrag_m": np.array([0.0, 0.0, 0.0])},
            "aus_gewerbebetrieb": {"betrag_m": np.array([0.0, 0.0, 0.0])},
            "aus_kapitalvermögen": {"kapitalerträge_m": np.array([0.0, 0.0, 0.0])},
            "aus_nichtselbstständiger_arbeit": {
                "bruttolohn_m": np.array([6000.0, 4000.0, 0.0])
            },
            "aus_selbstständiger_arbeit": {"betrag_m": np.array([0.0, 0.0, 0.0])},
            "aus_vermietung_und_verpachtung": {"betrag_m": np.array([0.0, 0.0, 0.0])},
            "ist_hauptberuflich_selbstständig": np.array([False, False, False]),
            "sonstige": {
                "alle_weiteren_m": np.array([0.0, 0.0, 0.0]),
                "rente": {
                    "betriebliche_altersvorsorge_m": np.array([0.0, 0.0, 0.0]),
                    "geförderte_private_vorsorge_m": np.array([0.0, 0.0, 0.0]),
                    "sonstige_private_vorsorge_m": np.array([0.0, 0.0, 0.0]),
                },
            },
        },
        "gemeinsam_veranlagt": np.array([True, True, False]),
    },
    "arbeitslosengeld_2": {
        "p_id_einstandspartner": np.array([1, 0, -1])
    },  # not required for ESt, but we need it later
    "familie": {
        "alleinerziehend": np.array([False, False, False]),
        "p_id_ehepartner": np.array([1, 0, -1]),
        "p_id_elternteil_1": np.array([-1, -1, 0]),
        "p_id_elternteil_2": np.array([-1, -1, 1]),
    },
    "geburtsjahr": np.array([1985, 1985, 2020]),
    "kindergeld": {
        "in_ausbildung": np.array([False, False, False]),
        "p_id_empfänger": np.array([-1, -1, 0]),
    },
    "p_id": np.array([0, 1, 2]),
    "hh_id": np.array([0, 0, 0]),
    "sozialversicherung": {
        "kranken": {"beitrag": {"privat_versichert": np.array([False, False, False])}},
        "pflege": {"beitrag": {"hat_kinder": np.array([True, True, False])}},
        "rente": {
            "altersrente": {"betrag_m": np.array([0.0, 0.0, 0.0])},
            "erwerbsminderung": {"betrag_m": np.array([0.0, 0.0, 0.0])},
            "jahr_renteneintritt": np.array([2060, 2060, 2090]),
        },
    },
}

The status quo is the following:

In [ ]:
main(
    main_target=MainTarget.results.df_with_nested_columns,
    policy_date_str="2025-01-01",
    input_data=InputData.tree(INPUT_DATA_TREE),
    tt_targets={"tree": {"einkommensteuer": {"betrag_y_sn": None}}},
    include_warn_nodes=False,
)

## Modifying Parameters

GETTSIM's parameters are stored in different objects depending on the form they are
specified in the law. If you modify the parameters in the `policy_environment`, you will
encounter the following objects:

1. **ScalarParam**: A scalar parameter, i.e. a parameter that is a single number.
2. **DictParam**: A parameter that is a flat dictionary with homogeneous keys and values
   (i.e. all keys and values are of the same type).
3. **ConsecutiveIntLookupTableParam**: A lookup table that stores values and assigns a
   consecutive integer index to each value.
4. **PiecewisePolynomialParam**: A piecewise polynomial parameter, i.e. a parameter that
    describes a piecewise polynomial function.
5. **RawParam**: A parameter that does not fit into the other categories. For these
   parameters, we need `ParamFunction`s to process them (see next section).

All of those parameter classes have the following attributes:
- `leaf_name`: The leaf name of the parameter in GETTSIM's policy environment.
- `start_date`: The date from which the parameter is valid (if applicable).
- `end_date`: The date until which the parameter is valid (if applicable).
- `unit`: The unit of the parameter (if applicable).
- `reference_period`: The period over which the parameter is valid (if applicable).
- `name`: The name of the parameter.
- `description`: A more elaborate description of the parameter.
- `value`: The value of the parameter.
- `note`: Some notes (if applicable).
- `reference`: A legal reference.

When modifying parameters, you will mostly care about the `value` attribute.

### Scalar Parameters

Scalar parameters are the simplest type of parameters. They are represented by the
`ScalarParam` class.

Let's take a look at the `arbeitnehmerpauschbetrag` parameter, a flat deduction applied
to income from regular employment.

As you can see the `arbeitnehmerpauschbetrag` parameter is a `ScalarParam` object and
its value is 1230€ in the status quo.

In [ ]:
status_quo_environment["einkommensteuer"]["einkünfte"][
    "aus_nichtselbstständiger_arbeit"
]["arbeitnehmerpauschbetrag"]

In [ ]:
status_quo_environment["einkommensteuer"]["einkünfte"][
    "aus_nichtselbstständiger_arbeit"
]["arbeitnehmerpauschbetrag"].value

Let's increase the parameter.

#### Step 1: Create a copy of the status quo policy environment. 

This is good practice to avoid in-place modifications of the original policy
environment.

In [ ]:
higher_arbeitnehmerpauschbetrag_pe = copy_environment(status_quo_environment)

#### Step 2: Create the new parameter.

Create a new `ScalarParam` object. To do this, we first import the `ScalarParam` class
from GETTSIM and then instantiate it with the new value.

**Note**: You don't have to specify all attributes of the `ScalarParam` class. Only the
value attribute is required.

In [ ]:
from gettsim.tt import ScalarParam

higher_arbeitnehmerpauschbetrag = ScalarParam(value=1600)

#### Step 3: Replace the old parameter with the new one in the new policy environment

In [ ]:
higher_arbeitnehmerpauschbetrag_pe["einkommensteuer"]["einkünfte"][
    "aus_nichtselbstständiger_arbeit"
]["arbeitnehmerpauschbetrag"] = higher_arbeitnehmerpauschbetrag

Let's call GETTSIM with the modified policy environment.

In [ ]:
main(
    main_target=MainTarget.results.df_with_nested_columns,
    policy_date_str="2025-01-01",
    input_data=InputData.tree(INPUT_DATA_TREE),
    tt_targets={"tree": {"einkommensteuer": {"betrag_y_sn": None}}},
    policy_environment=higher_arbeitnehmerpauschbetrag_pe,
    include_warn_nodes=False,
)

### Dict Parameters

Dict parameters are parameters that are a flat dictionary with homogeneous keys and
values. They are represented by the `DictParam` class. They are stored as a flat
dictionary in the `policy_environment`.

Let's take a look at the `parameter_kinderfreibetrag` parameter. This parameter
contains tax deductions for parents of young children.

As you can see the `parameter_kinderfreibetrag` parameter is a `DictParam` object and
its value is a dictionary.

In [ ]:
status_quo_environment["einkommensteuer"]["parameter_kinderfreibetrag"]

In [ ]:
status_quo_environment["einkommensteuer"]["parameter_kinderfreibetrag"].value

Let's modify the parameter by increasing the "sächliches Existenzminimum" for children.
We follow the same steps as in the previous section.

In [ ]:
from gettsim.tt import DictParam

# Step 1: Create a copy of the status quo policy environment.
higher_kinderfreibetrag_pe = copy_environment(status_quo_environment)

# Step 2: Create a new parameter `parameter_kinderfreibetrag`
higher_kinderfreibetrag = DictParam(
    value={
        "betreuung_erziehung_ausbildung": 1464,
        "sächliches_existenzminimum": 4000,
    },
)

# Step 3: Insert the new parameter into the copied policy environment
higher_kinderfreibetrag_pe["einkommensteuer"]["parameter_kinderfreibetrag"] = (
    higher_kinderfreibetrag
)

main(
    main_target=MainTarget.results.df_with_nested_columns,
    policy_date_str="2025-01-01",
    input_data=InputData.tree(INPUT_DATA_TREE),
    tt_targets={"tree": {"einkommensteuer": {"betrag_y_sn": None}}},
    policy_environment=higher_kinderfreibetrag_pe,
    include_warn_nodes=False,
)

### Consecutive Int Lookup Table Parameters

In many cases, we need to look up values in a table, for which we tend to use
dictionaries. E.g., the increase in the normal retirement age for the 1940s cohort would
look like this:

```python
{
    1940: 65,
    1941: 65,
    1942: 65,
    1943: 65,
    1944: 65,
    1945: 65,
    1946: 65,
    1947: 65.08333333,
    1948: 65.16666667,
    1949: 65.25,
}
``` 

For all its efficiency, our alternative computing engine Jax does not support such lookup tables. 
We thus construct our own version, which only works for consecutive integer values
(i.e., the dictionary keys could not be `red`, `blue`, `green`, or  `1`, `7`, `9`, etc.).

Let's look at the implementation.

In [ ]:
status_quo_environment["sozialversicherung"]["rente"]["altersrente"][
    "regelaltersrente"
]["altersgrenze_gestaffelt"]

The `value` attribute of `ConsecutiveIntLookupTableParam` stores a `ConsecutiveIntLookupTableParamValue` object:

In [ ]:
status_quo_environment["sozialversicherung"]["rente"]["altersrente"][
    "regelaltersrente"
]["altersgrenze_gestaffelt"].value

This object has a `look_up` method, which returns the value for the given key.

In [ ]:
for i in range(1940, 1950):
    print(
        i,
        status_quo_environment["sozialversicherung"]["rente"]["altersrente"][
            "regelaltersrente"
        ]["altersgrenze_gestaffelt"].value.look_up(i),
    )

Let's create a modified version with a steeper increase in the normal retirement age.

In order to instantiate a `ConsecutiveIntLookupTableParamValue`, we have a function
`get_consecutive_int_lookup_table_param_value` that takes a dictionary as above:

In [ ]:
from gettsim.tt import (
    ConsecutiveIntLookupTableParam,
    get_consecutive_int_lookup_table_param_value,
)

# Step 1: Create a copy of the status quo policy environment.
increased_nra_by_birth_year_pe = copy_environment(status_quo_environment)

# Step 2: Create a new parameter `altersgrenze_gestaffelt`
increased_nra_by_birth_year = ConsecutiveIntLookupTableParam(
    value=get_consecutive_int_lookup_table_param_value(
        raw={
            1940: 65,
            1941: 65,
            1942: 65,
            1943: 65,
            1944: 65,
            1945: 65.5,
            1946: 66,
            1947: 66.5,
            1948: 67,
            1949: 67,
        },
        xnp=np,
    ),
)

# Step 3: Insert the new parameter into the copied policy environment
increased_nra_by_birth_year_pe["sozialversicherung"]["rente"]["altersrente"][
    "regelaltersrente"
]["altersgrenze_gestaffelt"] = increased_nra_by_birth_year

main(
    main_target=MainTarget.results.df_with_nested_columns,
    policy_date_str="2025-01-01",
    input_data=InputData.tree(
        {
            "geburtsjahr": np.array([1944, 1945, 1946, 1947, 1948]),
            "p_id": np.array([0, 1, 2, 3, 4]),
        }
    ),
    tt_targets={
        "tree": {
            "geburtsjahr": None,
            "sozialversicherung": {
                "rente": {"altersrente": {"regelaltersrente": {"altersgrenze": None}}}
            },
        }
    },
    policy_environment=increased_nra_by_birth_year_pe,
    include_warn_nodes=False,
)

### Piecewise Polynomial Parameters

Piecewise polynomial parameters specify a continuous polynomial (first to third degree)
on the real line. GETTSIM uses them whenever a parameter is a function of a continuous
variable (like income, age, etc.).

Let's take a look at `parameter_behindertenpauschbetrag`, a tax deduction for disabled
individuals. The deduction is a function of the degree of disability (between 0 and
100).

In [ ]:
status_quo_environment["einkommensteuer"]["abzüge"]["parameter_behindertenpauschbetrag"]

The `value` attribute of `PiecewisePolynomialParam` stores a `PiecewisePolynomialParamValue` object:

In [ ]:
status_quo_environment["einkommensteuer"]["abzüge"][
    "parameter_behindertenpauschbetrag"
].value

Let's change the schedule such that any disability degree above the level of 50% gets a
flat deduction of 3000€.

In order to instantiate a `PiecewisePolynomialParamValue`, we have a function
`get_piecewise_parameters` that takes a configuration dictionary and processes it. Here,
we define a `piecewise_constant` function on the real line with two pieces:

- `0`: `-inf` to `0` with `0` as intercept
- `1`: `0` to `inf` with `3000` as intercept

**Note:** Piecewise polynomial parameters must **always** be defined on the real line.
**Note:** It can be complex to build the `parameter_dict` for
`get_piecewise_parameters`. For more complex schedules, take a look at the tutorial on
piecewise polynomial functions [not available yet, add link eventually; in the mean
time, look at the implementation in GETTSIM's parameter yamls (or ask for help)].

In [ ]:
from gettsim.tt import PiecewisePolynomialParam, get_piecewise_parameters

increased_behindertenpauschbetrag = PiecewisePolynomialParam(
    value=get_piecewise_parameters(
        func_type="piecewise_constant",
        parameter_dict={
            0: {
                "lower_threshold": "-inf",
                "intercept_at_lower_threshold": 0,
            },
            1: {
                "lower_threshold": 0.5,
                "upper_threshold": "inf",
                "intercept_at_lower_threshold": 3000.0,
            },
        },
        leaf_name="parameter_behindertenpauschbetrag",
        xnp=np,
    )
)

increased_behindertenpauschbetrag_pe = copy_environment(status_quo_environment)
increased_behindertenpauschbetrag_pe["einkommensteuer"]["abzüge"][
    "parameter_behindertenpauschbetrag"
] = increased_behindertenpauschbetrag

main(
    main_target=MainTarget.results.df_with_nested_columns,
    policy_date_str="2025-01-01",
    input_data=InputData.tree(INPUT_DATA_TREE),
    tt_targets={"tree": {"einkommensteuer": {"betrag_y_sn": None}}},
    policy_environment=increased_behindertenpauschbetrag_pe,
    include_warn_nodes=False,
)

### Raw params

Raw parameters are parameters that are stored in a way that doesn't fit into the other
categories. `RawParam`s need to be processed via a `ParamFunction` to make them usable
for standard `ColumnFunction`s (see next section).

Because of this, `RawParam`s may come in very different shapes. One example is the
income tax tariff. GETTSIM stores it as it is reported in the law. To receive the shape
we need to wrap it in a `PiecewisePolynomialParam`, some post-processing is needed to
convert it to the shape we need.

Let's take a look at the `raw_parameter_einkommensteuertarif` parameter.

In [ ]:
status_quo_environment["einkommensteuer"]["raw_parameter_einkommensteuertarif"]

In [ ]:
status_quo_environment["einkommensteuer"]["raw_parameter_einkommensteuertarif"].value

We modify it by increasing the income exempt from any income tax (upper threshold of the
first bracket) from 12096€ to 14000€.

In [ ]:
from gettsim.tt import RawParam

# Step 1: Create a copy of the status quo policy environment.
increased_tax_exemption_pe = copy_environment(status_quo_environment)

# Step 2: Create the new parameter.
increased_tax_exemption = RawParam(
    value={
        0: {
            "lower_threshold": "-inf",
            "upper_threshold": 14000,
            "rate_linear": 0,
            "rate_quadratic": 0,
            "intercept_at_lower_threshold": 0,
        },
        1: {"upper_threshold": 17443, "rate_linear": 0.14},
        2: {"upper_threshold": 68480, "rate_linear": 0.2397},
        3: {"upper_threshold": 277825, "rate_linear": 0.42, "rate_quadratic": 0},
        4: {"upper_threshold": "inf", "rate_linear": 0.45, "rate_quadratic": 0},
    }
)

# Step 3: Insert the new parameter into the copied policy environment
increased_tax_exemption_pe["einkommensteuer"]["raw_parameter_einkommensteuertarif"] = (
    increased_tax_exemption
)

main(
    main_target=MainTarget.results.df_with_nested_columns,
    policy_date_str="2025-01-01",
    input_data=InputData.tree(INPUT_DATA_TREE),
    tt_targets={"tree": {"einkommensteuer": {"betrag_y_sn": None}}},
    policy_environment=increased_tax_exemption_pe,
    include_warn_nodes=False,
)

## Modifying Parameter Functions

Parameter functions process parameters. Often, they process `RawParam`s to make them
usable for standard `ColumnFunction`s, but they can also be used to modify the standard
parameters described above. Parameter functions must not depend on `ColumnFunction`s.

Let's take a look at the `kinderfreibetrag_pro_kind_y` parameter. It is created via a
`ParamFunction` by summing up the values of the `parameter_kinderfreibetrag` parameter
dictionary.

```python
@param_function()
def kinderfreibetrag_pro_kind_y(parameter_kinderfreibetrag: dict[str, float]) -> float:
    return sum(parameter_kinderfreibetrag.values())
```

In [ ]:
status_quo_environment["einkommensteuer"]["kinderfreibetrag_pro_kind_y"]

Let's look at a reform that modifies the tax deduction for children by adding the
"Bildung und Teilhabe" bonus of the "Kinderzuschlag" to it. This effectively increases
the tax deduction for parents with children by 348€ per child. 

Instead of rewriting the underlying parameters (strictly speaking, the parameters
themselves are not affected by this reform), we can use a `ParamFunction` to modify the
tax deduction for children only.

The original function looks like this:

```python
@param_function()
def kinderfreibetrag_pro_kind_y(
    parameter_kinderfreibetrag: dict[str, float],
) -> float:
    return sum(parameter_kinderfreibetrag.values())
```

This is our modified version:

In [ ]:
from gettsim.tt import param_function


@param_function()
def kinderfreibetrag_pro_kind_y(
    parameter_kinderfreibetrag: dict[str, float],
    kinderzuschlag__parameter_existenzminimum: dict[str, float],
) -> float:
    return (
        sum(parameter_kinderfreibetrag.values())
        + kinderzuschlag__parameter_existenzminimum["bildung_und_teilhabe"]["kind"]
    )

As before, we create a copy of the status quo environment and replace the old function
with the new one.

In [ ]:
bildung_und_teilhabe_tax_deductible_pe = copy_environment(status_quo_environment)
bildung_und_teilhabe_tax_deductible_pe["einkommensteuer"][
    "kinderfreibetrag_pro_kind_y"
] = kinderfreibetrag_pro_kind_y

main(
    main_target=MainTarget.results.df_with_nested_columns,
    policy_date_str="2025-01-01",
    input_data=InputData.tree(INPUT_DATA_TREE),
    tt_targets={"tree": {"einkommensteuer": {"betrag_y_sn": None}}},
    policy_environment=bildung_und_teilhabe_tax_deductible_pe,
    include_warn_nodes=False,
)

## Modifying Column Objects

`ColumnObject`s are objects that operate on columns of data. The policy environment
contains three types of `ColumnObject`s:

- `PolicyFunction`s: The most common type of `ColumnObject`. They perform the standard
  tax and transfer calculations on the individual (or group) level.
- `GroupCreationFunction`s: These are used to create grouping IDs for individuals (e.g.
  Bedarfsgemeinschaften or Ehegemeinschaften).
- `AggByGroupFunction`s: These functions aggregate the results of `PolicyFunction`s
  by group (e.g. aggregate a transfer from the individual level to the level of a
  Bedarfsgemeinschaft).
- `AggByPIDFunction`s: These functions aggregate the results of `PolicyFunction`s by
  person (e.g. aggregate a transfer from the individual level to the level of one
  specific parent).
- `PolicyInput`s: These are placeholders for the basic inputs of the tax and transfer
  system, delivering information about the input type. If you add new inputs when
  modifying the policy environemnt **and** want to create input data templates (via
  `MainTarget.templates.input_data_dtypes`), you should add a `PolicyInput` for each new
  input variable. Else, you can safely ignore them.


Here, we look at how to modify `PolicyFunction`s. If you want to specify aggregation
functions, take a look at [GEP
2](https://gettsim.readthedocs.io/en/stable/geps/gep-04.html).

### Policy Functions

`PolicyFunction`s are usually written to operate on rows of the input data. They can
take any parameter or other `ColumnObject` as inputs.

**Note**: Some `PolicyFunction`s operate on columns of input data. Their decorator will
contain the term `vectorization_strategy="not_required"`.

Let's take a look at the tax deduction for children again. We look at a reform that
abolishes the tax deduction for families with less than 2 children.

This is the original function:

```python
@policy_function()
def kinderfreibetrag_y(
    anzahl_kinderfreibeträge: int,
    kinderfreibetrag_pro_kind_y: float,
) -> float:
    """Individual child allowance."""
    return kinderfreibetrag_pro_kind_y * anzahl_kinderfreibeträge
```

To do this, we do, as above, the following steps:
1. Create a copy of the status quo environment
2. Create a new parameter `min_anzahl_kinder_für_kinderfreibetrag`
3. Create a new `PolicyFunction` that modifies the tax deduction for children based on
   the new parameter
4. Insert the new parameter and function into the copy of the status quo environment

In [ ]:
# Step 1: Create a copy of the status quo environment
tax_deduction_only_for_at_least_two_children_pe = copy_environment(
    status_quo_environment
)

# Step 2: Create a new parameter `min_anzahl_kinder_für_kinderfreibetrag`
from gettsim.tt import ScalarParam

min_anzahl_kinder_für_kinderfreibetrag = ScalarParam(value=2)

# Step 3: Create a new `PolicyFunction` that modifies the tax deduction for children
# based on the new parameter
from gettsim.tt import policy_function


@policy_function()
def kinderfreibetrag_y(
    anzahl_kinderfreibeträge: int,
    kinderfreibetrag_pro_kind_y: float,
    familie__anzahl_kinder_fg: int,
    min_anzahl_kinder_für_kinderfreibetrag: int,
) -> float:
    """Individual child allowance."""
    if familie__anzahl_kinder_fg >= min_anzahl_kinder_für_kinderfreibetrag:
        out = kinderfreibetrag_pro_kind_y * anzahl_kinderfreibeträge
    else:
        out = 0.0
    return out


# Step 4: Add the new parameter and function to the policy environment
tax_deduction_only_for_at_least_two_children_pe["einkommensteuer"][
    "kinderfreibetrag_y"
] = kinderfreibetrag_y
tax_deduction_only_for_at_least_two_children_pe["einkommensteuer"][
    "min_anzahl_kinder_für_kinderfreibetrag"
] = min_anzahl_kinder_für_kinderfreibetrag


main(
    main_target=MainTarget.results.df_with_nested_columns,
    policy_date_str="2025-01-01",
    input_data=InputData.tree(INPUT_DATA_TREE),
    tt_targets={"tree": {"einkommensteuer": {"betrag_y_sn": None}}},
    policy_environment=tax_deduction_only_for_at_least_two_children_pe,
    include_warn_nodes=False,
)